In [3]:
import os 
import sys

MODEL = "llama3.2:1b"

In [4]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)
model.invoke("Tell me a joke about AI")

C:\Users\DELL\AppData\Local\Temp\ipykernel_19640\1034660084.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model=MODEL)
C:\Users\DELL\AppData\Local\Temp\ipykernel_19640\1034660084.py:6: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=MODEL)


'A robot asked a human, "Why did you create me?"\n\nThe human replied, "I\'m afraid I don\'t know. Why?"'

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a brief funny history/n")

'Here\'s a brief, lighthearted history that might tickle your funny bone:\n\nHave you ever heard the story of the origin of the fart joke? Well, it all started in ancient Greece, around 500 BCE. The Greeks believed that flatulence was a sign of divine displeasure, and anyone who let out a toot could be struck down by the gods.\n\nThe first recorded instance of fart jokes dates back to 1908, when a writer named Joseph Wood Krutch published a humorous article in The New Yorker called "Fart: A Conversation with the Gods." In it, he joked that the only way to deal with the impending doom that was World War I was to just let out a good fart.\n\nThe modern fart joke, however, is often attributed to Ernest Hemingway, who allegedly wrote an entire novel, "Death in the Afternoon," about the joys of being hit by a perfect fart during a hunting trip. Unfortunately for Hemingway, his literary masterpiece was more known for its gruesome content than its clever writing.\n\nAnd then there\'s the stor

In [14]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t answer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("2025 Budget Performance.pdf")
pages = loader.load_and_split()
pages

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)


[Document(metadata={'producer': 'macOS Version 10.14.6 (Build 18G9323) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20241218144408Z00'00'", 'title': 'Microsoft Word - Document2', 'moddate': "D:20241218144408Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': '2025 Budget Performance.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content="PRESENTATION BY HIS EXCELLENCY , PRESIDENT BOLA AHMED TINUBU, GCFR OF THE 2025 BUDGET OF RESTORATION: SECURING PEACE, REBUILDING PROSPERITY TO A JOINT SESSION OF THE NATIONAL ASSEMBLY ON WEDNESDAY 18TH DECEMBER, 2024   PROTOCOLS: -- The Vice President, Senator Kashim She5ma -- Senate President, Dis7nguished Senator Godswill Akpabio -- Right Honourable Speaker, Tajudeen Abbas -- Na7onal Chairman of our great party, Excellency, Abdullahi Umar Ganduje -- State Governors here present -- Dis7nguished Leaders and Members of the Na7onal Assembly -- Senior Government Oﬃcials here present -- Gentlemen of the press, My Fellow Nig

In [11]:
from langchain_community.vectorstores import DocArrayInMemorySearch 

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [12]:
retriever = vectorstore.as_retriever()
retriever.invoke('Agenda')

[Document(metadata={'producer': 'macOS Version 10.14.6 (Build 18G9323) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20241218144408Z00'00'", 'title': 'Microsoft Word - Document2', 'moddate': "D:20241218144408Z00'00'", 'keywords': '', 'aapl:keywords': '[]', 'source': '2025 Budget Performance.pdf', 'total_pages': 6, 'page': 2, 'page_label': '3'}, page_content='13. These clear results of gradual recovery, among others, reﬂect the resilience of our economy and the impact of deliberate policy choices we made from the outset.  2024 BUDGET PERFORMANCE  14. I am happy to inform this Na1onal Assembly that our administra1on abained remarkable milestones in implemen1ng the 2024 Budget. In 2024, we achieved: o 14.55 trillion naira in revenue, mee1ng 75 percent of our target as of the third quarter. o 21.60 trillion naira in expenditure, represen1ng 85 percent of our target, also in the third quarter.  15. While challenges persist, we improved revenue collec1on and fulﬁlled key obliga1o

In [15]:
from operator import itemgetter
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt 
    | model
    | parser
)

In [18]:
questions = [
    "What is the budget for 2025?",
    "What are the main objectives of the budget?",
    "How much is allocated for education?",
    "What is the total budget amount?",
    "Are there any new initiatives in the budget?"
]
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the budget for 2025?
Answer: The budget for 2025 is referred to as "Budget of Restraint: Securing Peace, Rebuilding Prosperity" in the document.

Question: What are the main objectives of the budget?
Answer: Based on the document, the main objectives of the 2025 Budget Proposal are:

1. To restore macroeconomic stability.
2. To enhance the business environment.
3. To foster inclusive growth, employment, and poverty reduction.
4. To promote equitable income distribution and human capital development.

These objectives are mentioned in section 16 of the document, which outlines the policy direction and goals of the 2025 Budget Proposal.

Question: How much is allocated for education?
Answer: According to the document, 826.90 billion naira is allocated for infrastructure development in the educa1onal sector in the 2025 Budget proposal. Additionally, 402 billion naira is allocated for infrastructure investments in the health sector in the 2025 Budget. There are also provi

In [ ]:
print('hello world')